# WCA Results - Future Competitions
Created by Michael George (AKA Logiqx)

Link: https://www.speedsolving.com/forum/showthread.php?54128-How-fast-are-the-over-40-s-in-competitions

In [1]:
# from EventsLib import *

## Read Known Persons from CSV

Read person data from CSV into memory, prior to processing

In [2]:
import os, csv, datetime
import urllib.request, ssl
import json
from bs4 import BeautifulSoup

class Person:
    
    def __init__(self, fields):
        """Initialisise the person"""
        
        self.id = fields[0]
        self.name = fields[1]
        self.country = fields[2]
        self.username = fields[3]
        self.usernum = int(fields[4])
        self.ageCategory = int(fields[5])
        self.highlight = fields[6]
        self.userId = fields[7]
        if self.highlight == 'Y':
            self.name = '* ' + self.name + ' *'


    def getWcaLink(self, eventId = None):
        if (eventId):
            link = '<a href="https://www.worldcubeassociation.org/persons/%s#%s">%s</a>' % (self.id, eventId, self.name)
        else:
            link = '<a href="https://www.worldcubeassociation.org/persons/%s">%s</a>' % (self.id, self.name)

        return link


    def getSpeedsolvingLink(self):
        if self.usernum > 0:
            profile = '%s.%d' % (self.username.replace(' ', '-').replace('.', '-').lower(), self.usernum)
            link = '<a href="https://www.speedsolving.com/members/%s">%s</a>' % (profile, self.username)
        else:
            link = ''

        return link


class Competition:
    
    def __init__(self, ageCategory = 40):
        """Initialisise the partial results"""
        
        self.persons = {}
        self.users = {}
        self.attendees = []
        self.ageCategory = ageCategory


    def readPersons(self, basename):
        """Read seniors from CSV into memory"""
        
        self.persons = {}

        # Read rows using the CSV reader
        fn = os.path.join('..', 'data', 'public', basename + '.csv')
        with open(fn, 'r') as f:
            csvReader = csv.reader(f)
            
            # Process each row individually
            for line in csvReader:
                person = Person(line)

                if person.ageCategory >= self.ageCategory and person.id not in self.persons:
                    self.persons[person.id] = person
                    self.users[int(person.userId)] = person


    def getAttendees(self, fields):
        """Download the list of attendees"""

        def parseHtml(st):
            soup = BeautifulSoup(st, "lxml")
            anchors = soup.find_all("a")

            for anchor in anchors:
                href = anchor.get("href")
                if href:
                    personId = ''
                    if 'worldcubeassociation.org/persons/' in href:
                        personId = href[-10:]
                    elif 'worldcubeassociation.org/results/p.php?i=' in href:
                        personId = href[-10:]
                    elif 'cubingchina.com/results/person/' in href:
                        personId = href[-10:]

                    if id and personId in self.persons.keys():
                        self.attendees.append(self.persons[personId])

        def parseJson(st):           
            jsonTxt = response.read().decode('utf-8')
            registrations = json.loads(jsonTxt)

            for registration in registrations:
                userId = registration['user_id']
                if (userId in self.users.keys()):
                    self.attendees.append(self.users[userId])

                    
        self.compId = fields[0]
        self.compName = fields[1]
        self.compCity = fields[2]
        self.compCountry = fields[3]
        self.compWebsite = fields[4]
        self.startDate = fields[5]
        self.endDate = fields[6]       

        self.attendees = []

        comp_url = ''
        if 'zawody4event.pl' in self.compWebsite:
            comp_url = self.compWebsite
        elif 'speedcubing.pl' in self.compWebsite:
            comp_url = self.compWebsite
        elif 'cubingchina.com' in self.compWebsite:
            comp_url = self.compWebsite + '/competitors'
        elif 'cubing-tw.net' in self.compWebsite:
            comp_url = self.compWebsite + '/competitors'
        elif 'cubecomp.de' in self.compWebsite:
            comp_url = self.compWebsite + '/competitors'
        elif 'speedcubing.nz' in self.compWebsite:
            comp_url = self.compWebsite + '/competitors'
        elif 'canadiancubing.com' in self.compWebsite:
            comp_url = self.compWebsite + '/Competitors'
            
        if comp_url:
            req = urllib.request.Request(comp_url, headers={'User-Agent': 'Mozilla'})
            response = urllib.request.urlopen(req)
            parseHtml(response)
         
        else:
            comp_url = "https://www.worldcubeassociation.org/api/v0/competitions/%s/registrations" % self.compId

            req = urllib.request.Request(comp_url, headers={'User-Agent': 'Mozilla'})
            response = urllib.request.urlopen(req)
            parseJson(response)

        # Output status
        print('Processed %s - %d seniors' % (self.compName, len(self.attendees)))


    def listAttendees(self, weekNo):
        """List seniors from memory"""

        if len(self.attendees) > 0:
            link = '<a href="https://www.worldcubeassociation.org/competitions/%s">%s</a>' % (self.compId, self.compName)
            startDate = datetime.datetime.strptime(self.startDate, '%Y-%m-%d')
            endDate = datetime.datetime.strptime(self.endDate, '%Y-%m-%d')
            if startDate.day == endDate.day:
                dates = startDate.strftime('%b %-d, %Y')
            elif startDate.month == endDate.month:
                dates = startDate.strftime('%b %-d') + endDate.strftime(' - %-d, %Y')
            elif startDate.year == endDate.year:
                dates = startDate.strftime('%b %-d') + endDate.strftime(' - %b %-d, %Y')
            else:
                dates = startDate.strftime('%b %-d, %Y') + endDate.strftime(' - %b %-d, %Y')

            if weekNo == 1:
                html = '<details open>\n'
            else:
                html = '<details>\n'
            html += '  <summary>%s - %s, %s - %s</summary>\n' % (link, self.compCity, self.compCountry, dates)
            html += '  <p style="margin-left: 18px">\n'

            count = 0
            for person in sorted(self.attendees, key=lambda person: person.name):
                if count:
                    html += '<br/>\n'

                ageSuffix = ', %d+' % int(person.ageCategory) if int(person.ageCategory) > self.ageCategory else ''
                html += '    %s, %s%s' % (person.getWcaLink(), person.country, ageSuffix)
                if person.usernum > 0:
                    html += ' - %s on Speedsolving.com' % person.getSpeedsolvingLink()
                
                count += 1

            html += '\n'
            html += '  </p>\n'
            html += '</details>\n\n'
        else:
            html = ''
            
        return html

## Analyse Competitions

Process the competitions one-by-one

In [3]:
import datetime

fn = 'Future_Competitions.md'
with open(os.path.join('..', 'templates', fn), 'r') as f:
    html = ''.join(f.readlines())

refreshed = datetime.datetime.now().replace(microsecond=0).isoformat().replace('T', ' ')
html += 'Last refreshed: ' + refreshed + ' (UTC)\n\n'

fn = 'known_senior_details'
competition = Competition()
competition.readPersons(fn)

fn = os.path.join('..', 'data', 'public', 'future_competitions.csv')
with open(fn, 'r') as f:
    csvReader = csv.reader(f)

    prevDate = ''
    weekNo = 0
    for line in csvReader:
        competition.getAttendees(line)

        if len(competition.attendees) > 0:
            endDate = datetime.datetime.strptime(competition.endDate, '%Y-%m-%d')
            endDate += datetime.timedelta(days = 6 - (endDate.weekday() + 6) % 7)
            if endDate != prevDate:
                html += '<h3>%s</h3>\n\n' % endDate.strftime('%b %-d, %Y')
                prevDate = endDate
                weekNo += 1

            html += competition.listAttendees(weekNo)

fn = 'Future_Competitions.md'
with open(os.path.join('..', 'docs', fn), 'w') as f:
    f.write(html)

print('Future Competitions updated!')

Processed CubingUSA Nationals 2019 - 5 seniors
Processed German Nationals 2019 - 7 seniors
Processed Breaking Records Huancayo 2019 - 0 seniors
Processed Colégio Asther Open 2019 - 1 seniors
Processed Egyptian Championship 2019 - 0 seniors
Processed Taizhou Open 2019 - 0 seniors
Processed Tarija Open 2019 - 0 seniors
Processed CIC Pasto Open 2019 - 0 seniors
Processed Ha Noi Championship 2019 - 0 seniors
Processed Horse City 2019 - 0 seniors
Processed İstanbul Summer 2019 - 0 seniors
Processed SSL 3 Varberg 2019 - 2 seniors
Processed Urnieta Open 2019 - 0 seniors
Processed West Visayas Championship 2019 - 0 seniors
Processed Baku Summer 2019 - 0 seniors
Processed Dongguan Summer 2019 - 0 seniors
Processed Sucursal Del Cielo 2019 - 0 seniors
Processed Suzhou Open 2019 - 0 seniors
Processed Michigan 2019 - 1 seniors
Processed Austin Summer 2019 - 1 seniors
Processed Los Ángeles al Cubo 2019 - 0 seniors
Processed Matsu Open 2019 - 0 seniors
Processed Puget Sound Summer 2019 - 2 seniors
Pr

## All Done!